<a href="https://colab.research.google.com/github/affan1317/GEOL0069-EOYA/blob/main/Fetching_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source of data
We are utilising the Copernicus Data Space together with Google Earth Engine, which provide extensive satellite imageries and geospatial data. It is also freely accessible, fostering open science and research. Google Earth Engine can be accessed with the normal Google account most people would already have, while The Copernicus Data Space needs a separate registration, which can be done via the [website](https://dataspace.copernicus.eu/).

First, we mount our Google Drive to allow easy and direct access to our files. Again, by using Google Colab together with Google Drive, local memory and storage would not be the limiting factor.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Fetching Logic
The process of data fetching is meticulously structured to ensure we efficiently retrieve accurate and comprehensive data. The logic underlying this process involves several key steps:

1. Area and Time Specification: Initially, we define the geographical scope and the specific time frame of interest. This precise specification allows us to target our data retrieval effectively.

2. Retrieving File Names from Google Earth Engine: Once the area and time parameters are set, we proceed to fetch a list of relevant file names from Google Earth Engine. This platform provides a robust interface for identifying datasets that match our specified criteria.

3. Converting to Copernicus Filename Format: After obtaining the list of file names from Google Earth Engine, the next step involves transforming these names into the format recognised by the Copernicus Dataspace. This conversion is crucial for ensuring compatibility and facilitating the subsequent data retrieval process.

4. Fetching Raw Data from Copernicus Dataspace: With the correctly formatted file names in hand, we then access the Copernicus Dataspace to retrieve the raw data. It's important to note that Google Earth Engine does not provide raw data in the same format as the Copernicus Dataspace. Hence, this step is essential for obtaining the data in its native, unaltered format.

# Initial set up
*   Install the required packages
*   Authenticate with Google Earth



In [ ]:
import numpy as np
from shapely.geometry import Polygon, Point
import ee
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
import subprocess

ee.Authenticate()
ee.Initialize(project = 'geol0069week3fetchingdata')

## Read in the functions needed including:
* Parsing GEE filename for Sentinel-2 and Sentinel-3
* Quering Sentinel-2 and Sentinel-3 data based on extracted info
* Converting GEE filename to Copernicus filename
* Retrieving access token for Copernicus Data Space
* Downloading the queried data

In [ ]:
def parse_gee_filename_s2(gee_filename):
    """
    Parses the Google Earth Engine filename to extract satellite name, sensing date, and start time.

    Parameters:
    gee_filename (str): Filename obtained from Google Earth Engine.

    Returns:
    tuple: Contains satellite name, sensing date, and start time.
    """
    parts = gee_filename.split('_')
    sensing_date = parts[0]
    tile_number = parts[2]
    return sensing_date, tile_number

def parse_gee_filename_s3(gee_filename):
    """
    Parses the Google Earth Engine filename to extract satellite name, sensing date, and start time.

    Parameters:
    gee_filename (str): Filename obtained from Google Earth Engine.

    Returns:
    tuple: Contains satellite name, sensing date, and start time.
    """
    parts = gee_filename.split('_')
    satellite = parts[0] + '_OL_1_EFR'
    start_datetime = parts[1]
    end_datetime = parts[2]

    # Extract date from the start_datetime (assuming the format is like '20180601T014926')
    sensing_date = start_datetime[:8]
    start_time = start_datetime[9:]

    return satellite, sensing_date, start_time

def get_access_token(username, password):
    """
    Retrieves access token from Copernicus Dataspace using the provided credentials.

    Parameters:
    username (str): Username for Copernicus Dataspace.
    password (str): Password for Copernicus Dataspace.

    Returns:
    str: Access token for authenticated sessions.
    """
    url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
    data = {
        'grant_type': 'password',
        'username': username,
        'password': password,
        'client_id': 'cdse-public'
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    return response.json()['access_token']

def query_sentinel2_data(sensing_start_date, tile_number, token):
    """
    Queries the Sentinel-2 data from the Copernicus Data Space based on sensing start date, tile number, and access token.

    Parameters:
    sensing_start_date (str): The start date and time for the data sensing in the format 'YYYYMMDDTHHMMSS'.
    tile_number (str): The specific tile number of the Sentinel-2 data to be queried.
    token (str): The access token for authenticating requests to the Copernicus Data Space.

    Returns:
    DataFrame: A DataFrame containing the query results with details about the Sentinel-2 data.

    The function constructs a query URL with specified parameters, sends a request to the Copernicus Data Space,
    and returns the results as a DataFrame. It filters the data based on the tile number and the content start date
    within a certain time window.
    """
    # Convert sensing_start_date to datetime object and format it for the query
    start_time = datetime.strptime(sensing_start_date, '%Y%m%dT%H%M%S')
    end_time = start_time + timedelta(hours=2)  # Adjust the time window as necessary
    start_time_str = start_time.strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%SZ')

    # Construct the request URL with the contains function for tile number
    url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'{tile_number}') and Collection/Name eq 'SENTINEL-2' and ContentDate/Start gt {start_time_str} and ContentDate/Start lt {end_time_str}"
    headers = {'Authorization': f'Bearer {token}'}

    # Make the API request
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    return pd.DataFrame.from_dict(response.json()['value'])

def query_sentinel3_olci_data(satellite, sensing_date, start_time, token):
    """
    Queries Sentinel-3 OLCI data from Copernicus Data Space based on satellite name, sensing date, and start time.

    Parameters:
    satellite (str): Name of the satellite.
    sensing_date (str): Date of the data sensing.
    start_time (str): Start time of the data sensing.
    token (str): Access token for authentication.

    Returns:
    DataFrame: A DataFrame containing the query results with details about the Sentinel-3 OLCI data.
    """
    # Convert sensing_date to datetime object and format it for the query
    sensing_datetime = datetime.strptime(f'{sensing_date}T{start_time}', '%Y%m%dT%H%M%S')
    sensing_datetime = sensing_datetime - timedelta(seconds=1)

    # Construct the request URL using the filter structure provided
    url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter=contains(Name,'{satellite}') and "
        f"ContentDate/Start ge {sensing_datetime.strftime('%Y-%m-%dT%H:%M:%S.000Z')} and "
        f"ContentDate/Start le {(sensing_datetime + timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S.000Z')}&"
        f"$orderby=ContentDate/Start&$top=1000"
    )
    headers = {'Authorization': f'Bearer {token}'}

    # Print the URL for debugging
    print(url)

    # Make the API request
    response = requests.get(url, headers=headers)
    # Check if the request was successful
    if response.status_code != 200:
        # Print error details and return an empty DataFrame if the request failed
        print(f"Error: Unable to fetch data. Status Code: {response.status_code}. Response: {response.text}")
        return pd.DataFrame()

    # Convert the JSON response to a DataFrame
    search_results_df = pd.DataFrame.from_dict(response.json()['value'])

    # Convert the 'ContentDate/Start' to datetime objects and sort the results
    search_results_df['SensingStart'] = pd.to_datetime(search_results_df['ContentDate'].apply(lambda x: x['Start']))
    search_results_df.sort_values(by='SensingStart', inplace=True)

    return search_results_df

def extract_correct_product_name(df, start_time, tile_number):
    """
    Extracts the correct product name and ID from a dataframe based on a specific start time and tile number.

    Parameters:
    df (DataFrame): The dataframe containing product information.
    start_time (str): The start time used to filter the products.
    tile_number (str): The tile number used to filter the products.

    Returns:
    tuple: A tuple containing the first matching product name and ID, or (None, None) if no match is found.
    """
    # Adjusted regex pattern to match the filename format
    pattern = f'MSIL1C.*{start_time}.*_{tile_number}_'
    filtered_products = df[df['Name'].str.contains(pattern, regex=True)]


    # Return the first matching product name, or None if not found
    return filtered_products['Name'].iloc[0] if not filtered_products.empty else None, filtered_products['Id'].iloc[0] if not filtered_products.empty else None

def process_image_pair(s2_ee_image_id, token):
    """
    Processes a pair of Sentinel-2 images by querying the Copernicus Data Space to find the corresponding product name and ID.

    Parameters:
    s2_ee_image_id (str): The Sentinel-2 Earth Engine image ID.
    token (str): The access token for authenticating requests to the Copernicus Data Space.

    Returns:
    tuple: A tuple containing the product name and ID for the corresponding Sentinel-2 image.
    """
    sensing_start_date = s2_ee_image_id.split('_')[0]
    tile_number = s2_ee_image_id.split('_')[2]

    # Query the Copernicus Data Space
    df = query_sentinel2_data(sensing_start_date, tile_number, token)

    # Extract the correct MSIL1C product name
    return extract_correct_product_name(df, sensing_start_date, tile_number)

def fetch_S3_images_by_area_and_date(date_range, spatial_extent, area_of_interest):
    """
    Fetches Sentinel-3 OLCI images based on a specified date range and area of interest.

    :param date_range: List containing the start and end dates (e.g., ['2018-06-01', '2018-06-02'])
    :param spatial_extent: List containing the spatial extent [min_lon, min_lat, max_lon, max_lat]
    :param area_of_interest: ee.Geometry object defining the specific area for which to fetch images

    :return: List of dictionaries, each containing details about a fetched image, including its ID, date, and download URL.
    """
    # Initialize the Earth Engine module
    ee.Initialize()

    # Define variables for Sentinel-3 OLCI query
    S3_product = 'COPERNICUS/S3/OLCI'

    # Query for Sentinel-3 data within the specified date range and area of interest
    S3_collection = ee.ImageCollection(S3_product) \
        .filterDate(date_range[0], date_range[1]) \
        .filterBounds(area_of_interest)

    # Convert S3_collection to a list of image IDs
    S3_image_ids = S3_collection.aggregate_array('system:index').getInfo()
    S3_images_info = S3_collection.getInfo()['features']

    # Initialize an empty list to store details
    S3_image_details = []

    # Iterate through each image in the collection
    for img_info in S3_images_info:
        # Fetch image ID
        image_id = img_info['id']

        # Fetch image date and other properties as needed
        image_date = img_info['properties']['system:time_start']  # Example property

        # Append the details to the list
        S3_image_details.append({
            'id': image_id,
            'date': image_date
        })

    return S3_image_details

def download_single_product(product_id, file_name, access_token, download_dir="downloaded_products"):
    """
    Download a single product from the Copernicus Data Space.

    :param product_id: The unique identifier for the product.
    :param file_name: The name of the file to be downloaded.
    :param access_token: The access token for authorization.
    :param download_dir: The directory where the product will be saved.
    """
    # Ensure the download directory exists
    os.makedirs(download_dir, exist_ok=True)

    # Construct the download URL
    url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"

    # Set up the session and headers
    headers = {"Authorization": f"Bearer {access_token}"}
    session = requests.Session()
    session.headers.update(headers)

    # Perform the request
    response = session.get(url, headers=headers, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Define the path for the output file
        output_file_path = os.path.join(download_dir, file_name + ".zip")

        # Stream the content to a file
        with open(output_file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded: {output_file_path}")
    else:
        print(f"Failed to download product {product_id}. Status Code: {response.status_code}")

# Sentinel-2 Optical Image
The data S2 data can be visualised in the [Copernicus Browser](https://browser.dataspace.copernicus.eu/?zoom=8&lat=74.67164&lng=-68.05701&themeId=DEFAULT-THEME&visualizationUrl=https%3A%2F%2Fsh.dataspace.copernicus.eu%2Fogc%2Fwms%2Fa91f72b5-f393-4320-bc0f-990129bd9e63&datasetId=S2_L2A_CDAS&fromTime=2024-03-04T00%3A00%3A00.000Z&toTime=2024-03-04T23%3A59%3A59.999Z&layerId=1_TRUE_COLOR&demSource3D=%22MAPZEN%22&cloudCoverage=30&dateMode=SINGLE).
Be sure to change the gee_filename to the one you desire. In this project, we are looking at the ocean betwen Greenland and Canada on the date (04/03/2024). Note that the sensing time is at 171211.

In [ ]:
gee_filename = '20240304T171211_20240304T203849_T19XEC'           # change accordingly
file_name, product_id = process_image_pair(gee_filename, token)

# Get access token for Copernicus Data Space (ensure your credentials are correct)
username = 'affankb1317@gmail.com'      # be sure to enter your own credentials
password = '4ffanMazlan_'
token = get_access_token(username, password)
access_token = token

print(file_name)
print(product_id)

download_dir = '/content/drive/MyDrive/GEOL0069/EOYA'     # replace with your directory
download_single_product(product_id, file_name, access_token, download_dir)

S2A_MSIL1C_20240304T171211_N0510_R112_T19XEC_20240304T192251.SAFE
e3fe8a77-3fa4-40e4-937f-2b925d3449b2


## Colocated Sentinel-3 OLCI data
With the acquired Sentinel-2 data, the Sentinel-3 OLCI data is obtained for the same date (04/03/2024), just under two hours apart (155608).

In [ ]:
gee_image_id = 'S3B_20240304T155608_20240304T155711'

# Parse the GEE filename to get the date and time
satellite, sensing_date, start_time = parse_gee_filename_s3(gee_image_id)

# Get access token for Copernicus Data Space (ensure your credentials are correct)
username ='affankb1317@gmail.com'
password ='4ffanMazlan_'
token = get_access_token(username, password)

# Query the Copernicus Data Space for the corresponding Sentinel-3 OLCI data
s3_olci_data = query_sentinel3_olci_data(satellite, sensing_date, start_time, token)

# Assuming s3_olci_data is a DataFrame or a dictionary containing the query results
# Print the first filename and id from the list, which is usually the one we want
print(s3_olci_data['Name'][0])
print(s3_olci_data['Id'][0])

https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'S3B_OL_1_EFR') and ContentDate/Start ge 2024-03-04T15:56:07.000Z and ContentDate/Start le 2024-03-05T15:56:07.000Z&$orderby=ContentDate/Start&$top=1000
S3B_OL_1_EFR____20240304T155608_20240304T155711_20240305T084636_0063_090_211_1620_PS2_O_NT_003.SEN3
af0704af-c08f-428e-9bc5-b93735db2336


In [ ]:
download_dir = "/content/drive/MyDrive/GEOL0069/EOYA"  # Replace with your desired download directory
product_id = s3_olci_data['Id'][0]
file_name = s3_olci_data['Name'][0]
# Download the single product
download_single_product(product_id, file_name, access_token, download_dir)

## Sentinel-3 Altimetry Data
The next set of functions is to find the altimetry data associated with the Sentinel-3 OLCI data obtained from the above steps.

In [ ]:
import requests
import pandas as pd
import subprocess
import os
import time
import shutil
import json
from datetime import date
from joblib import Parallel, delayed
import zipfile
import sys
import glob
import numpy as np

def get_access_token(username, password):
    """
    Obtain an access token to the Copernicus Data Space Ecosystem.
    Necessary for the download of hosted products.
    """
    p =  subprocess.run(f"curl --location --request POST 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token' \
            --header 'Content-Type: application/x-www-form-urlencoded' \
            --data-urlencode 'grant_type=password' \
            --data-urlencode 'username={username}' \
            --data-urlencode 'password={password}' \
            --data-urlencode 'client_id=cdse-public'", shell=True,capture_output=True, text=True)
    access_dict = json.loads(p.stdout)
    return access_dict['access_token'], access_dict['refresh_token']

#=============================================================================================================================================================#

def get_new_access_token(refresh_token):
    """
    Obtain a new access token to the Copernicus Data Space Ecosystem using a previously provided refesh token.
    """
    p =  subprocess.run(f"curl --location --request POST 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token' \
    --header 'Content-Type: application/x-www-form-urlencoded' \
    --data-urlencode 'grant_type=refresh_token' \
    --data-urlencode 'refresh_token={refresh_token}' \
    --data-urlencode 'client_id=cdse-public'", shell=True,capture_output=True, text=True)
    access_dict = json.loads(p.stdout)
    return access_dict['access_token'], access_dict['refresh_token']

#=============================================================================================================================================================#

def get_S3_SI_search_results_df(date):
    """
    Obtain a pandas dataframe of Sentinel-3 sea ice thematic products for a given date.
    """
    json = requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'SENTINEL-3' and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'SR_2_LAN_SI') and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'timeliness' and att/OData.CSC.StringAttribute/Value eq 'NT') and ContentDate/Start gt {(date-pd.Timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%SZ')} and ContentDate/End lt {(date+pd.Timedelta(days=2)).strftime('%Y-%m-%dT%H:%M:%SZ')}&$top=1000").json()

    results_df = pd.DataFrame.from_dict(json['value'])
    results_df['Satellite'] = [row['Name'][:3] for i,row in results_df.iterrows()]
    results_df['SensingStart'] = [pd.to_datetime(row['ContentDate']['Start']) for i,row in results_df.iterrows()]
    results_df['SensingEnd'] = [pd.to_datetime(row['ContentDate']['End']) for i,row in results_df.iterrows()]
    results_df =  results_df[(results_df['SensingEnd'] >= date) & (results_df['SensingStart'] <= date+pd.Timedelta(days=1))]
    results_df = results_df.sort_values(by='SensingStart')
    return results_df

#=============================================================================================================================================================#

def filter_duplicate_products_versions(results_df, keep_latest=True ):
    """
    Filter Sentinel-3 product dataframe to remove duplicate verions of files.
    By default, we keep the latest version of the file. E.g., where an operation version
    and a reprocessed version exists, we keep the reprocessed version.
    """
    results_df['name_snippet'] = [row['Name'][:47] for i,row in results_df.iterrows()]
    if  keep_latest == True:
        keep='last'
    else:
        keep = 'first'

    results_df = (
        results_df
        .sort_values(by='ModificationDate')
        .drop_duplicates(subset=['name_snippet'], keep=keep)
        .drop(columns = ['name_snippet'])
        .sort_values(by='SensingStart')
    )

    return results_df

def find_overlapping_sar(olci_filename, search_results_df):
    # Extract date and time from OLCI filename
    parts = olci_filename.split('_')
    olci_date_time = datetime.strptime(parts[7], '%Y%m%dT%H%M%S')

    # Filter SAR filenames based on overlapping criteria
    # This is a placeholder logic, adjust according to your specific criteria
    overlapping_sar = search_results_df[search_results_df['Name'].apply(lambda x: 'S3' in x and 'SR_2_LAN_SI' in x)]

    return overlapping_sar


def get_date_from_olci_filename(olci_filename):
    """
    Extracts the date from an OLCI filename.

    Parameters:
    olci_filename (str): The OLCI filename.

    Returns:
    datetime.date: The date extracted from the filename.
    """
    parts = olci_filename.split('_')
    date_str = parts[7][:8]  # Extract date part and truncate to YYYYMMDD format
    return pd.to_datetime(date_str, format='%Y%m%d').date()

def get_overlapping_sar_file(olci_filename, get_S3_SI_search_results_df, token):
    olci_date = get_date_from_olci_filename(olci_filename)
    start_date = olci_date - pd.Timedelta(days=1)
    end_date = olci_date + pd.Timedelta(days=1)
    dates = pd.date_range(start_date, end_date)

    all_overlapping_sar = pd.DataFrame()  # Collect all overlapping SAR files

    for date in dates:
        date = date.tz_localize('UTC')
        search_results_df = get_S3_SI_search_results_df(date)

        if search_results_df.empty:
            print(f"No SAR data found for date: {date}")
            continue

        filtered_df = filter_duplicate_products_versions(search_results_df)
        if filtered_df.empty:
            print(f"No SAR data after filtering for date: {date}")
            continue

        overlapping_sar = find_overlapping_sar(olci_filename, filtered_df)
        if not overlapping_sar.empty:
            all_overlapping_sar = pd.concat([all_overlapping_sar, overlapping_sar], ignore_index=True)

    return all_overlapping_sar

from datetime import datetime

def check_overlap(row, olci_filename, olci_start, olci_end):
    """
    Checks if the SAR file's sensing period overlaps with the OLCI file's sensing period and if it's from the same satellite.

    Parameters:
    row (Series): A row from the SAR search results DataFrame.
    olci_filename (str): The OLCI filename.
    olci_start (datetime): Start time of OLCI sensing period.
    olci_end (datetime): End time of OLCI sensing period.

    Returns:
    bool: True if there's an overlap and the satellite is consistent, False otherwise.
    """
    # Extract satellite identifier from the OLCI filename
    satellite = olci_filename.split('_')[0]  # e.g., S3A or S3B

    # Parse SAR start and end times
    sar_start = datetime.strptime(row['ContentDate']['Start'], '%Y-%m-%dT%H:%M:%S.%fZ')
    sar_end = datetime.strptime(row['ContentDate']['End'], '%Y-%m-%dT%H:%M:%S.%fZ')

    # Check for temporal overlap and satellite consistency
    is_temporal_overlap = sar_start <= olci_end and sar_end >= olci_start
    is_same_satellite = satellite in row['Name']

    return is_temporal_overlap and is_same_satellite

# Adjust the find_overlapping_sar function to include the OLCI filename in the check_overlap call
def find_overlapping_sar(olci_filename, search_results_df):
    # Extract date and time from OLCI filename
    parts = olci_filename.split('_')
    olci_sensing_start = datetime.strptime(parts[7], '%Y%m%dT%H%M%S')
    olci_sensing_end = datetime.strptime(parts[8], '%Y%m%dT%H%M%S')

    # Filter for SAR files that overlap with the OLCI sensing period
    overlapping_sar = search_results_df[search_results_df.apply(lambda row: check_overlap(row, olci_filename, olci_sensing_start, olci_sensing_end), axis=1)]

    return overlapping_sar


Download Sentinel 3 Altimetry Data

In [ ]:
username ='affankb1317@gmail.com'
password ='4ffanMazlan_'

token = get_access_token(username, password)
token, refresh_token = get_access_token(username, password)

olci_filename = 'S3B_OL_1_EFR____20240304T155608_20240304T155711_20240305T084636_0063_090_211_1620_PS2_O_NT_003.SEN3' # replace with the one you are interested in.
overlapped_df = get_overlapping_sar_file(olci_filename, get_S3_SI_search_results_df, token)
product_id = overlapped_df['Id'].iloc[0]
file_name = overlapped_df['Name'].iloc[0]
download_dir = '/content/drive/MyDrive/GEOL0069/EOYA'
download_single_product(product_id, file_name, token, download_dir)  # altimetry data

# Downloaded products
Now, we have three different folders downloaded in this notebook, which includes:
1.   Sentinel-2 Optical Image
2.   Colocated Sentinel-3 OLCI Data
3.   Corresponding Sentinel-3 Altimetry Data

